In [10]:
### IMPORT DANYCH ###
import pandas as pd
import itertools
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from statistics import mean
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

danesas=pd.read_csv('danesasowe.csv',delimiter=';',decimal=',')
danesas=danesas.drop(['account_id','datazalozenia',
                'dataurodzenia','datazalozenia',
                'datapozyczki','status','disp_id'
                ,'typkarty','duration','payments','balance'],axis=1)

#Wybor kolumn ktore moga wejsc do modelu
kolumnyX=["platnosci","wielkosckredytu",
          "ileremittancetoanotherbank","ilewithdraw",
          "ilecollectionfromanotherbank","iletransakcji",
          "staz_pozyczka","ilecreditcardwithdrawal"]
X=danesas
y=X['Czy_problemy']

#Wszystkie mozliwe kombinacje kolumn - sprawdzenie ktore z kolumn daja najlepszy wynik
listapodzbiorowzmiennych=[]
for L in range(0, len(kolumnyX)+1):
    for subset in itertools.combinations(kolumnyX, L):
        listapodzbiorowzmiennych.append(list(subset)) 
#Klasyfikacja        
def Przeprowadz_Klasyfikacje(klasyfikatory,zmienna,X,y):
    Xt=X[zmienna]
    Xt=(Xt-Xt.mean())/Xt.std()
    #Xt=(Xt-Xt.min())/(Xt.max()-Xt.min())
    wartoscacctrain=[]
    wartoscacctest=[]
    wartoscroctrain=[]
    wartoscroctest=[]
    for key, classifier in klasyfikatory.items(): 
        for i in range(10):
            X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2,stratify=y.values)
            model = classifier.fit(X_train, y_train)
            #########################################
            step_factor = 0.05 
            threshold_value = 0.2 
            roc_score=0
            predicted_proba = model.predict_proba(X_train) #probability of prediction
            while threshold_value <=0.85: #continue to check best threshold upto probability 0.8
                temp_thresh = threshold_value
                predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
                #print('Threshold',temp_thresh,'--',roc_auc_score(y_test, predicted))
                if roc_score<roc_auc_score(y_train, predicted): #store the threshold for best classification
                    roc_score = roc_auc_score(y_train, predicted)
                    thrsh_score = threshold_value
                threshold_value = threshold_value + step_factor
            #print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)
            ######################################
            y_pred_proba=model.predict_proba(X_train)
            y_pred = (y_pred_proba[:,1] >= thrsh_score).astype(bool)
            #########################################
            step_factor = 0.05 
            threshold_value = 0.2 
            roc_score=0
            predicted_proba = model.predict_proba(X_test) #probability of prediction
            while threshold_value <=0.85: #continue to check best threshold upto probability 0.8
                temp_thresh = threshold_value
                predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
                #print('Threshold',temp_thresh,'--',roc_auc_score(y_test, predicted))
                if roc_score<roc_auc_score(y_test, predicted): #store the threshold for best classification
                    roc_score = roc_auc_score(y_test, predicted)
                    thrsh_score = threshold_value
                threshold_value = threshold_value + step_factor
            #print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)
            ######################################
            y_pred1=(model.predict_proba(X_test)[:,1]>=thrsh_score).astype(bool)
            
            
            
            training_score = cross_val_score(model, X_train, y_train, cv=5)
            test_score = cross_val_score(model, X_test, y_test, cv=5)
            wartoscacctrain.append(round(training_score.mean(), 2))
            wartoscacctest.append(round(test_score.mean(), 2))
            wartoscroctrain.append(roc_auc_score(y_train, y_pred))
            wartoscroctest.append(roc_auc_score(y_test, y_pred1))
        print(key,zmienna)
        print("Dane treningowe")
        print(classification_report(y_train, y_pred))
        print(confusion_matrix(y_train,y_pred))
        print()
        print("Dane testowe")
        print(classification_report(y_test, y_pred1))
        print(confusion_matrix(y_test,y_pred1))
        print("AUC TRAIN:",round(mean(wartoscroctrain),2),"AUC TEST:",round(mean(wartoscroctest),2))
        print("ACC TRAIN:",round(mean(wartoscacctrain),2),"ACC TEST:",round(mean(wartoscacctest),2))
        print()

    
        

In [7]:
#Zwykła klasyfikacja - zastosowanie jedynie wag#
import numpy as np
classifiers = {
    "LogisiticRegression": LogisticRegression(max_iter=30000,class_weight="balanced"),
    "DecisionTreeClassifier": DecisionTreeClassifier(class_weight="balanced"),
    "Support Vector Classifier": SVC(kernel='rbf',probability=True,class_weight="balanced"),
    "RandomForestClassifier": RandomForestClassifier(class_weight="balanced")
}
Przeprowadz_Klasyfikacje(classifiers,listapodzbiorowzmiennych[-1],X,y)


LogisiticRegression ['platnosci', 'wielkosckredytu', 'ileremittancetoanotherbank', 'ilewithdraw', 'ilecollectionfromanotherbank', 'iletransakcji', 'staz_pozyczka', 'ilecreditcardwithdrawal']
Dane treningowe
              precision    recall  f1-score   support

           0       0.95      0.60      0.73       484
           1       0.19      0.77      0.31        61

    accuracy                           0.62       545
   macro avg       0.57      0.68      0.52       545
weighted avg       0.87      0.62      0.69       545

[[289 195]
 [ 14  47]]

Dane testowe
              precision    recall  f1-score   support

           0       0.95      0.80      0.87       122
           1       0.29      0.67      0.40        15

    accuracy                           0.78       137
   macro avg       0.62      0.73      0.63       137
weighted avg       0.88      0.78      0.82       137

[[97 25]
 [ 5 10]]
AUC TRAIN: 0.69 AUC TEST: 0.7
ACC TRAIN: 0.66 ACC TEST: 0.69

DecisionTreeClassifie

In [8]:
# UPSAMPLING #
import numpy as np
df_minority=X[X['Czy_problemy']==1]
df_majority=X[X['Czy_problemy']==0]
df_minority_upsampled=resample(df_minority,replace=True,n_samples=len(df_majority))
df_upsampled=pd.concat([df_minority_upsampled,df_majority])
y_upsampled=df_upsampled['Czy_problemy']
X_upsampled=df_upsampled.drop(['Czy_problemy'],axis=1)
classifiers = {
    "LogisiticRegression": LogisticRegression(max_iter=30000,class_weight="balanced"),
    "DecisionTreeClassifier": DecisionTreeClassifier(class_weight="balanced",max_depth=6,criterion='gini'),
    "Support Vector Classifier": SVC(kernel='rbf',probability=True,class_weight="balanced"),
    "RandomForestClassifier": RandomForestClassifier(class_weight="balanced")
}
Przeprowadz_Klasyfikacje(classifiers,listapodzbiorowzmiennych[-1],X_upsampled,y_upsampled)


LogisiticRegression ['platnosci', 'wielkosckredytu', 'ileremittancetoanotherbank', 'ilewithdraw', 'ilecollectionfromanotherbank', 'iletransakcji', 'staz_pozyczka', 'ilecreditcardwithdrawal']
Dane treningowe
              precision    recall  f1-score   support

           0       0.76      0.60      0.67       485
           1       0.67      0.81      0.73       484

    accuracy                           0.70       969
   macro avg       0.71      0.70      0.70       969
weighted avg       0.71      0.70      0.70       969

[[293 192]
 [ 94 390]]

Dane testowe
              precision    recall  f1-score   support

           0       0.69      0.64      0.66       121
           1       0.67      0.72      0.69       122

    accuracy                           0.68       243
   macro avg       0.68      0.68      0.68       243
weighted avg       0.68      0.68      0.68       243

[[77 44]
 [34 88]]
AUC TRAIN: 0.7 AUC TEST: 0.7
ACC TRAIN: 0.68 ACC TEST: 0.68

DecisionTreeClassifier

In [11]:
#SMOTE#
oversample = SMOTE(sampling_strategy=1,k_neighbors=5)
Xn, yn = oversample.fit_resample(X, y)
classifiers = {
    "LogisiticRegression": LogisticRegression(max_iter=30000,class_weight="balanced"),
    "DecisionTreeClassifier": DecisionTreeClassifier(class_weight="balanced",max_depth=6,criterion='gini'),
    "RandomForestClassifier": RandomForestClassifier(class_weight="balanced"),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(kernel='rbf',probability=True,class_weight="balanced")
}
Przeprowadz_Klasyfikacje(classifiers,listapodzbiorowzmiennych[-1],Xn,yn)

LogisiticRegression ['platnosci', 'wielkosckredytu', 'ileremittancetoanotherbank', 'ilewithdraw', 'ilecollectionfromanotherbank', 'iletransakcji', 'staz_pozyczka', 'ilecreditcardwithdrawal']
Dane treningowe
              precision    recall  f1-score   support

           0       0.80      0.58      0.67       484
           1       0.67      0.85      0.75       485

    accuracy                           0.72       969
   macro avg       0.73      0.72      0.71       969
weighted avg       0.73      0.72      0.71       969

[[280 204]
 [ 72 413]]

Dane testowe
              precision    recall  f1-score   support

           0       0.79      0.61      0.69       122
           1       0.68      0.83      0.75       121

    accuracy                           0.72       243
   macro avg       0.73      0.72      0.72       243
weighted avg       0.73      0.72      0.72       243

[[ 74  48]
 [ 20 101]]
AUC TRAIN: 0.72 AUC TEST: 0.73
ACC TRAIN: 0.7 ACC TEST: 0.7

DecisionTreeClassi

In [37]:
#Wlasny klasyfikator

def Wlasny_klasyfikator(klasyfikatory,zmienna,X,y):
    Xt=X[zmienna]
    Xt=(Xt-Xt.mean())/Xt.std()
    #Xt=(Xt-Xt.min())/(Xt.max()-Xt.min())
    wartoscacctrain=[]
    wartoscacctest=[]
    wartoscroctrain=[]
    wartoscroctest=[]
    predictprobatrain=[]
    predictprobatest=[]
    for key, classifier in klasyfikatory.items():
        X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2,stratify=y.values)
        model = classifier.fit(X_train, y_train)
        y_pred_proba=model.predict_proba(X_train)
        predictprobatrain.append(y_pred_proba)      
        
        y_pred_probatest=model.predict_proba(X_test)
        predictprobatest.append(y_pred_probatest)
        
    new_proba_train=sum(predictprobatrain,0)/len(klasyfikatory)
    new_proba_train=sum(predictprobatest,0)/len(klasyfikatory)
    y_pred = (y_pred_proba[:,1] >= 0.5).astype(bool)
    y_pred1=(model.predict_proba(X_test)[:,1]>=0.5).astype(bool)
    
    print("Dane treningowe")
    print(classification_report(y_train, y_pred))
    print(confusion_matrix(y_train,y_pred))
    print()
    print("Dane testowe")
    print(classification_report(y_test, y_pred1))
    print(confusion_matrix(y_test,y_pred1))
        
        
    
    

In [43]:
oversample = SMOTE(sampling_strategy=1,k_neighbors=5)
Xn, yn = oversample.fit_resample(X, y)
Xn=Xn
yn=yn
classifiers = {
    "LogisiticRegression": LogisticRegression(max_iter=30000,class_weight="balanced"),
    "KNearest": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Classifier": SVC(kernel='rbf',probability=True,class_weight="balanced")
}
Przeprowadz_Klasyfikacje1(classifiers,listapodzbiorowzmiennych[-1],Xn,yn)




Dane treningowe
              precision    recall  f1-score   support

           0       0.84      0.74      0.79       484
           1       0.77      0.86      0.81       485

    accuracy                           0.80       969
   macro avg       0.81      0.80      0.80       969
weighted avg       0.81      0.80      0.80       969

[[358 126]
 [ 66 419]]

Dane testowe
              precision    recall  f1-score   support

           0       0.90      0.73      0.81       122
           1       0.77      0.92      0.84       121

    accuracy                           0.82       243
   macro avg       0.83      0.82      0.82       243
weighted avg       0.84      0.82      0.82       243

[[ 89  33]
 [ 10 111]]
